In [1]:
using Roots
using DelimitedFiles
using LinearAlgebra
using NLsolve
using ApproximationGrids

In [2]:
############################################################
############################################################
global σ = [1, 0] #Radio de las macropartículas
global m = 2  #Número de especies total
#ϵ = 81.1  #Constante dieléctrica del agua
#Temp = 298 #Temperatura sin reducir
#e = 1.60217*10^(-19) #Carga eléctrica en C
#kB = 1.380649*10^(-23) #Constante de Boltzmann en J/K
global NKMAX = 4000

#Generamos vector
global kv = [ 0.01*i for i=1:NKMAX]
global dk = kv[2]-kv[1]

function save_data(nombre,formato)
    @assert typeof(nombre) == typeof("hola")
    open(nombre, "w") do io
        writedlm(io,formato)
    end
    println("Data saved as ", nombre)
end

save_data (generic function with 1 method)

In [3]:
#c(k) HS con PY-VW
function CHSVW(k,eta)
    #Corrección de Verlet-Weiss
    η=(eta-(1.0/16.0)*eta^2)
    σw=(η/eta)^(1.0/3.0)
    k=k*σw
    
    #Coeficientes para la c(r) propuesta por Wertheim
    α=-((1.0+2.0*η)^2.0)/(1.0-η)^4.0
    β=6.0*η*(((1.0+0.5*η)^2.0)/((1.0-η)^4.0))
    δ=(η/2.0)*α
    
    #Componentes de la C(k) (transformada de la c(r) propuesta por Wertheim)
    i1=(sin(k)-k*cos(k))/k^3.0
    i2=((-(k^2.0) + 2.0)*cos(k) + 2.0*k*sin(k)-2.0)/k^4.0
    i3=((-(k^4.0) + 12.0*(k^2.0) - 24)*cos(k) +(4.0*(k^3.0)-24.0*k)*sin(k) + 24.0)/k^6.0

    #Definiendo el desarrollo en serie de las i1,i2,i3 para k pequeñas
    ip1=(1.0/3.0)-((k^2.0)*(1.0/30.0))
    ip2=(1.0/4.0)-((k^2.0)*(1.0/36.0))
    ip3=(1.0/6.0)-((k^2.0)*(1.0/48.0))

    #C(k) 
    if k<0.075
        ck=24.0*η*(α*ip1+β*ip2+δ*ip3)
        else 
        ck=24.0*η*(α*i1+β*i2+δ*i3)
    end
    
    return ck

end

CHSVW (generic function with 1 method)

In [4]:
#FUNCIONES PARA HS+DY ATÉRMICO

#c(k) HS-DY atérmico con PY-VW
function CHSDYA(k,par)
    #par[5] es fracción en volumen
    K1=par[1]
    Z1=par[2]
    K2=par[3]
    Z2=par[4]
    η=par[5]
    k=k*σ[1]
    
    if K2==0||Z2==0
        RY=0
        RYp=0
    else
        RY=(24*η*K2)*((Z2*sin(k)+k*cos(k))/(k*(Z2^2+k^2)))
        RYp=(24*η*K2)*((Z2+1)/(Z2^2))
    end
    
    AY=(24*η*K1)*((Z1*sin(k)+k*cos(k))/(k*(Z1^2+k^2)))
    
    AYp=(24*η*K1)*((Z1+1)/(Z1^2))
    

    if k<0.075
        CTotal=CHSVW(k,par[5])+AYp-RYp
    else
        CTotal=CHSVW(k,par[5])+AY-RY
    end
    
    return CTotal
end

#S(k) HS-DY ATÉRMICO con PY-VW
function SHSDYA(Tr,Z1,K2,Z2,phi)
    K1=1/Tr
    parSS = [K1,Z1,K2,Z2,phi]
    CSS=zeros(NKMAX,1)
    SSS=zeros(NKMAX,1)
    
    for i=1:NKMAX
        CSS[i]=CHSDYA(kv[i],parSS)
        SSS[i]=1.0/(1.0 - CSS[i])
    end
    
    SSk=save_data("SKHSDYA_$(Tr)_$(Z1)_$(K2)_$(Z2)_$phi.dat", [kv SSS])

end

#Temperatura crítica para HS-DY ATÉRMICO con PY-VW
function TCHSDYA(Z1,K2,Z2)
    if K2==0||Z2==0
        fac2=0
    else
        fac2 = (Z2 + 1) / (Z2*Z2)
    end
    
    fac1 = (Z1 + 1) / (Z1*Z1)
        
    Tc = fac1 / (0.88815 + K2*fac2)
    
    println("Tc= ",Tc)
end

#Tsp para una phia
function TspinodalA(Z1,K2,Z2,phitest)        
    fac2 = (Z2 + 1) / (Z2*Z2)
    fac1 = (Z1 + 1) / (Z1*Z1)
    fac3 = 1/(24*phitest)
    fac4 = fac3*(1-CHSVW(0,phitest)) + K2*fac2
    Tspinodal= fac1 / fac4
    println("Tsp= ",Tspinodal)
    
    return Tspinodal
end

function SpinodalHSDYA(Z1,K2,Z2)
    PHIMAX = 1200
    phiv = [ 0.0005*i for i=1:PHIMAX]
    SSLO=zeros(PHIMAX,1)
    Sesp=0
    println("Curva espinodal HS+DY")
    
    for i=1:PHIMAX
        phitest=i*0.0005
        fac2 = (Z2 + 1) / (Z2*Z2)
        fac1 = (Z1 + 1) / (Z1*Z1)
        fac3 = 1/(24*phitest)
        fac4 = fac3*(1-CHSVW(0,phitest)) + K2*fac2
        Tspinodal= fac1 / fac4
        SSLO[i]=Tspinodal
    end
    
    SpinodalLO= save_data("SpHSDYA_$(Z1)_$(K2)_$(Z2).dat", [phiv SSLO])
end

#Tλ para una phia
function TlambdaA(Tc,Z1,K2,Z2,phia)
    Sesp=zeros(NKMAX,1)
    vi = [0,Z1,K2,Z2,phia]
    errortemp=1.0
    tol=1.0e-7
    limS=Tc+1
    limI=Tc
    Tla=1
    Ttest=1

    while errortemp>tol
        Ttest=(limS+limI)*0.5
        K1=1/Ttest
        vi[1]=K1
        flag=0

        for m=1:NKMAX
            Sesp[m]=1.0/(1.0 - CHSDYA(kv[m],vi))
        end
            
        mask=Sesp.<0
        cond=sum(mask)
        #println(cond," ",Ttest)
            
        if cond>0
            flag=1
        end

        if flag>=1
            limI=Ttest
        else
            limS=Ttest
        end

        errortemp=abs(1-limS/limI)
    end

    Tla=Ttest
    println("Tλ= ",Tla)
    
    return Tla
end

#λ HS-DY con PY-VW
function LambdaHSDYA(Ti,Z1,K2,Z2)
    PHIMAX = 120
    phiv = [ 0.005*i for i=1:PHIMAX]
    SSλLO=zeros(PHIMAX,1)
    Sesp=zeros(200,1)
    println("Lambda line HS+DY")
    tol=1.0e-7
    
    for i=1:PHIMAX
        phitest=i*0.005
        vi = [0,Z1,K2,Z2,phitest]
        errortemp=1.0
        limS=Ti
        limI=1.0e-8
        Ttest=1
        
        while errortemp>tol
            Ttest=(limS+limI)*0.5
            K1=1/Ttest
            vi[1]=K1
            flag=0
            
            for m=1:200
                Sesp[m]=1.0/(1.0 - CHSDYA(kv[m],vi))
            end
            mask=Sesp.<0
            cond=sum(mask)
            
            if cond>0
                Sesp[mask][1]
                flag=1
            end
            
            #=for m=1:100
                Sesp[m]=1.0/(1.0 - CHSDY(kv[m],vi))
                if Sesp[m]<=0.0
                    flag=flag+1
                    break
                end
            end=#
            
            if flag>=1
                limI=Ttest
            else
                limS=Ttest
            end

            errortemp=abs(1-limS/limI)
        end
        
        SSλLO[i]=Ttest
        

    end
    
    LambdaLO= save_data("LHSDYA_$(Z1)_$(K2)_$(Z2).dat", [phiv SSλLO])
 
end


LambdaHSDYA (generic function with 1 method)

In [5]:
#FUNCIONES PARA HS+DY DONDE T AFECTA AMBDAS INTERACCIONES

#c(k) HS-DY con PY-VW
function CHSDY(k,par)
    #par[5] es fracción en volumen
    K1=par[1]
    α=par[2]
    Z1=par[3]
    Z2=par[4]
    η=par[5]
    k=k*σ[1]
    
    if α==0||Z2==0
        RY=0
        RYp=0
    else
        RY=((Z2*sin(k)+k*cos(k))/(k*(Z2^2+k^2)))
        RYp=((Z2+1)/(Z2^2))
    end
    
    AY=((Z1*sin(k)+k*cos(k))/(k*(Z1^2+k^2)))
    
    AYp=(Z1+1)/(Z1^2)
    

    if k<0.075
        CTotal=CHSVW(k,par[5])+(24*η*K1)*(AYp-α*RYp)
    else
        CTotal=CHSVW(k,par[5])+(24*η*K1)*(AY-α*RY)
    end
    
    return CTotal
end

#S(k) HS-DY con PY-VW
function SHSDY(Tr,α,Z1,Z2,phi)
    K1=1/Tr
    parSS = [K1,α,Z1,Z2,phi]
    CSS=zeros(NKMAX,1)
    SSS=zeros(NKMAX,1)
    
    for i=1:NKMAX
        CSS[i]=CHSDY(kv[i],parSS)
        SSS[i]=1.0/(1.0 - CSS[i])
    end

    
    SSk=save_data("SKHSDY_$(Tr)_$(Z1)_$(K2)_$(Z2)_$phi.dat", [kv SSS])

end

#Temperatura crítica para HS-DY con PY-VW
function TCHSDY(α,Z1,Z2)
    if α==0||Z2==0
        fac2=0
    else
        fac2 = (Z2 + 1) / (Z2*Z2)
    end
    
    fac1 = (Z1 + 1) / (Z1*Z1)
    fac3= 3/2.65031
    Tc = (fac3)*(fac1-α*fac2)
    println("Tc= ",Tc)
end


function Tspinodal(Z1,K2,Z2,phitest)        
    fac2 = (Z2 + 1) / (Z2*Z2)
    fac1 = (Z1 + 1) / (Z1*Z1)
    fac3 = 1/(24*phitest)
    fac4 = fac3*(1-CHSVW(0,phitest)) + K2*fac2
    Tspinodal= fac1 / fac4
    println("Tsp= ",Tspinodal)
    
    return Tspinodal
end

function SpinodalHSDY(α,Z1,Z2)
    PHIMAX = 1200
    phiv = [ 0.0005*i for i=1:PHIMAX]
    SSLO=zeros(PHIMAX,1)
    Sesp=0
    println("Curva espinodal HS+DY")
    
    if α==0||Z2==0
        fac2=0
    else
        fac2 = (Z2 + 1) / (Z2*Z2)
    end
        
    fac1 = (Z1 + 1) / (Z1*Z1)
    
    for i=1:PHIMAX
        phitest=i*0.0005
        fac3 = 1/(24*phitest)
        fac4 = fac3*(1-CHSVW(0,phitest))
        Tspinodal= (fac1 - α*fac2) / fac4
        if Tspinodal<=0
            println("no hay spinodal")
            break
        end
        SSLO[i]=Tspinodal
    end
    
    SpinodalLO= save_data("SpHSDY_$(Z1)_$(K2)_$(Z2).dat", [phiv SSLO])
end


#Tλ para una phia
function Tlambda(Tc,α,Z1,Z2,phia)
    Sesp=zeros(500,1)
    vi = [0,α,Z1,Z2,phia]
    errortemp=1.0
    tol=1.0e-7
    limS=10
    limI=tol
    Tla=1
    Ttest=1

    while errortemp>tol
        Ttest=(limS+limI)*0.5
        K1=1/Ttest
        vi[1]=K1
        flag=0

        for m=1:500
            Sesp[m]=1.0/(1.0 - CHSDY(kv[m],vi))
        end
            
        mask=Sesp.<0
        cond=sum(mask)
        #println(cond," ",Ttest)
            
        if cond>0
            flag=1
        end

        if flag>=1
            limI=Ttest
        else
            limS=Ttest
        end

        errortemp=abs(1-limS/limI)
    end

    Tla=Ttest
    println("Tλ= ",Tla)
    
    return Tla
end

#λ HS-DY con PY-VW
function LambdaHSDY(Ti,α,Z1,Z2)
    PHIMAX = 120
    phiv = [ 0.005*i for i=1:PHIMAX]
    SSλLO=zeros(PHIMAX,1)
    Sesp=zeros(500,1)
    println("Lambda line HS+DY")
    tol=1.0e-7
    
    for i=1:PHIMAX
        phitest=i*0.005
        vi = [0,α,Z1,Z2,phitest]
        errortemp=1.0
        limS=10
        limI=tol
        Ttest=1
        
        while errortemp>tol
            Ttest=(limS+limI)*0.5
            K1=1/Ttest
            vi[1]=K1
            flag=0
            
            for m=1:500
                Sesp[m]=1.0/(1.0 - CHSDY(kv[m],vi))
            end
            mask=Sesp.<0
            cond=sum(mask)
            
            if cond>0
                Sesp[mask][1]
                flag=1
            end
            
            #=for m=1:100
                Sesp[m]=1.0/(1.0 - CHSDY(kv[m],vi))
                if Sesp[m]<=0.0
                    flag=flag+1
                    break
                end
            end=#
            
            if flag>=1
                limI=Ttest
            else
                limS=Ttest
            end

            errortemp=abs(1-limS/limI)
        end
        
        SSλLO[i]=Ttest
        

    end
    
    LambdaLO= save_data("LHSDY_$(α)_$(Z1)_$(Z2).dat", [phiv SSλLO])
 
end

LambdaHSDY (generic function with 1 method)

In [6]:
function λ(k)
    kmin=2*π*1.305/σ[1]
    lam=1/(1+(k/kmin)^2)
    λ=lam
    return λ
end

function γ(γ1,Si,Hi,rho)  
    suma1=0.0
    i=0

    for i=1:NKMAX
        A=(λ(kv[i])^2)*(Hi[i])^2
        B=(λ(kv[i])*Si[i])+((kv[i]^2)*γ1)
        C=λ(kv[i])+((kv[i]^2)*γ1)
        
        suma1 = suma1 + (kv[i]^4)*(A/(B*C))
        
    end
        #=recuerda que la renormalizacion es interna al factor de estructura, 
        y verlet weiss dice que la estructura correcta esta desfasada en terminos
        de la dimension de particula, lo que implica que tienes que renormalizar
        tanto k como la eta pero solo en s((k,eta)=#
    int1=suma1*dk/(6*rho*π^2)
    γ1= 1/int1
    
    return γ1
end

#Método iterativo de picard iteración de punto fijo
function criterioγ(Svec,Hvec,rho)
    i=0
    inf=1.0e8 
    tol=1.0e-7
    errorγ=1.0
    γtest=1.0e-7
    
    while tol<errorγ
        i=i+1  
        γp=γ(γtest,Svec,Hvec,rho) 
        γ1=γp
        errorγ=abs(1-γ1/γtest)
        γtest=γ1

        #println("γ= ",γtest," error= ",errorγ)
        #print(i)
        if (inf<γ1) break
        end 
        
        if i>10000
            break
        end
        
    end 
    
    
    return γtest
end

function arrestoHSDY(phitest,Z1,K2,Z2,Tlambda)
    tol=1.0e-7
    inf=1.0e8 
    Ttest=1
    errortemp=1
    limS=Tlambda+0.1
    limI=Tlambda
    Carr=zeros(NKMAX,1)
    Sarr=zeros(NKMAX,1)
    Harr=zeros(NKMAX,1)
    i=0
    
    while errortemp>tol
        i=i+1
        Ttest=(limS+limI)*0.5
        K1=1/Ttest
        rhotest=6*phitest/(π*(σ[1]^3))
        vi = [K1,Z1,K2,Z2,phitest]
        for i=1:NKMAX
            Carr[i]=CHSDY(kv[i],vi)
            Sarr[i]=1.0/(1.0 - Carr[i])
            Harr[i]=Sarr[i]-1.0
        end

        γ1=criterioγ(Sarr,Harr,rhotest)

        if γ1<inf
            limI=Ttest
        else
            limS=Ttest
        end

        errortemp=abs(1-limS/limI)
        
        if γ1<0
            println(Ttest)
            break
        end
        
        println("γ= ",γ1," T= ",Ttest)
    end
    
    return Ttest
end

arrestoHSDY (generic function with 1 method)

In [7]:
function SpinodalHSSW(Z1)
    PHIMAX = 1200
    phiv = [ 0.0005*i for i=1:PHIMAX]
    SSLO=zeros(PHIMAX,1)
    Sesp=0
    println("Curva espinodal HS+DY")
    
    for i=1:PHIMAX
        phitest=i*0.0005
        fac2 = 8*phitest*(Z1^3 - 1)
        #fac3 = 1/(24*phitest)
        fac4 = (1-CHSVW(0,phitest))
        Tspinodal= fac2 / fac4
        SSLO[i]=Tspinodal
    end
    
    SpinodalLO= save_data("SpHSSW_$(Z1).dat", [phiv SSLO])
end

Spinodal(1.5)

LoadError: UndefVarError: Spinodal not defined

In [43]:
#################################
#PARÁMETROS
#################################

ϕ = 0.31 #Fracción en volumen de las macropartículas
Tr = 0.55 #Temperatura reducida como T*=kT/K1 
K2 = 0.5
Z1= 10
Z2= 4

#SHSDYA(Tr,Z1,K2,Z2,ϕ)
SHSDY(Tr,α,Z1,Z2,ϕ)

Ti=10
#S(k) para HS+DY temperatura afecta ambas interacciones
#SHSDY(Tr,α,Z1,Z2,ϕ)
SpinodalHSDY(K2,Z1,Z2)
LambdaHSDY(Ti,K2,Z1,Z2)

#S(k) para HS+DY atérmico
#SHSDY(Tr,Z1,K2,Z2,ϕ)

#Spinodal y λ para HS+DY ATÉRMICO

SpinodalHSDYA(Z1,K2,Z2)
LambdaHSDYA(Ti,Z1,K2,Z2)

#Tc=Tspinodal(Ti,Z1,K2,Z2,ϕ)
#SHSDY(Tc,Z1,K2,Z2,ϕ)

#Tλ=Tlambda(Tc,Z1,K2,Z2,ϕ) #temperatura de línea λ en phia
#SHSDY(Tλ,Z1,K2,Z2,ϕ)
#Ta=arrestoHSDY(ϕ,Z1,K2,Z2,Tλ) #temperatura de arresto en phia

#dif=Ta-Tλ #diferencia en temperaturas de lambda y arresto en phia

#println(Z1," ",K2," ",Z2," ",dif," ",Ta," ",Tλ)

#difrel1=dif/Tλ
#println(difrel1)
#difrel2=dif/Tλ
#println(difrel2)
#difrel3=dif/Tλ
#println(difrel3)
#phia=0.15
#Tλ=Tlambda(0.1,Z1,K2,Z2,phia)
#Ta=arrestoHSDY(phia,Z1,K2,Z2,Tλ)
#println(Ta)
#=
#Tla=Tlambda(1,Z1,K2,Z2,ϕ)
for i=1:10
    Tla=0.07876+i*2e-5
    Tla=round(Tla, digits=5)
    SHSDY(Tla,Z1,K2,Z2,ϕ)
end


#S_HS=CHS(0,ϕ)
#Tcrit=Tcritica(ϕ,S_HS,Z1,Z2,K2)
#println(Tcrit)
#K1 = 3 #Amplitud Yukawa Atractiva (AY S-S)
#K1 = 1/Tr 
#Tr=1/K1
#Z1 = 2 #Alcance Yukawa Atractiva (AY S-S)
#K2 = 0.5 #Amplitud Yukawa Repulsiva (DLVO)
#Z2 = 0.5 #Alcanze Yukawa Repulsiva (DLVO)

##################################################################

#################################
#ESTRUCTURAS
#################################


##################################################################

#################################
#ESPINODALES
#################################

#=Curva espinodal HS+RY + perturbación AY - SS
Ti=0.1 #Temperatura inicial (T*) donde quieres empezar el barrido
phia=0.15=#
SpinodalHSDY(Ti,Z1,K2,Z2)
LambdaHSDY(Ti,Z1,K2,Z2)
Tλ=Tlambda(Ti,Z1,K2,Z2,phia)
Ta=arrestoHSDY(phia,Z1,K2,Z2,Tλ)
dif=Ta-Tλ 
println(Z1," ",K2," ",Z2," ",dif," ",Ta," ",Tλ)

##################################################################

#################################
#DIAGRAMAS DE ARRESTO
#################################

#Diagrama de arresto HS+RY + perturbación AY - SS
#Ti=2 #Temperatura inicial (T*) donde quieres empezar el barrido
#Tf=0.5 #Temperatura final (T*) donde quieres terminar el barrido
#DiagArresto(Ti,Tf,Z1,K2,Z2)
=#
##################################################################

Curva espinodal HS+DY
no hay spinodal
Data saved as SpHSDY_10_0.5_4.dat
Lambda line HS+DY
Data saved as LHSDY_0.5_10_4.dat
Curva espinodal HS+DY
Data saved as SpHSDYA_10_0.5_4.dat
Lambda line HS+DY
Data saved as LHSDYA_10_0.5_4.dat


In [42]:
TCHSDY(0.5,10,4)

Tc= -0.05235236632695798


In [16]:
#Z1=1
Z1=1
Tc=Tspinodal(Ti,Z1,K2,Z2,ϕ)
println("T spinodal ", Tc)
Tλ=Tlambda(Tc,Z1,K2,Z2,ϕ) #temperatura de línea λ en phia
println("T lambda ", Tλ)
Ta=arrestoHSDY(ϕ,Z1,K2,Z2,Tλ) #temperatura de arresto en phia
println("T arresto ", Ta)
dif=Ta-Tλ
println("Dif ", dif)
difrel=dif/Tλ
println("Dif rel ", difrel)
difpor=difrel*100
println("Dif por ", difpor)
#=
Z1=1
Ta=0.6733599626832605
Tλ=0.6733101332002283
Diferencia Ta-Tλ
dif=4.982948303222656e-5
Diferencia relativa dif/Tλ
difrel=7.400673267070573e-5
Diferencia porcentual difrel*100
difpor=0.007400673267070573
=#

Tsp= 0.23060369288539515
T spinodal 0.23060369288539515
Tλ= 0.23060808872794736
T lambda 0.23060808872794736
γ= 3.4840092533829465e9 T= 0.28060808872794735
γ= 1.649108096942972e11 T= 0.2556080887279474
γ= 1.370709051775166e13 T= 0.24310808872794737
γ= 8.995650089347895e8 T= 0.23685808872794736
γ= 4.30739756755624e10 T= 0.23373308872794735
γ= 2.1293037757115571e12 T= 0.23217058872794735
γ= 3.656096437623853e8 T= 0.23138933872794737
γ= 1.0836310254664417e8 T= 0.23099871372794736
γ= 6.964250837401618e9 T= 0.23080340122794735
γ= 1.7766331663186496e8 T= 0.23070574497794735
γ= 1872.8520430108836 T= 0.23065691685294737
γ= 1.8265438757567477e8 T= 0.23068133091544735
γ= 3.784657517257203e8 T= 0.23066912388419736
γ= 4.2387749790585856e9 T= 0.23066302036857236
γ= 4270.323353197569 T= 0.23065996861075988
γ= 1.3914639730663437e8 T= 0.2306614944896661
γ= 7000.3706837092195 T= 0.230660731550213
γ= 1.6605053922159452e9 T= 0.23066111301993955
γ= 9334.171440583159 T= 0.2306609222850763
γ= 1.621807673655

In [17]:
#Z1=10
Z1=10
Tc=Tspinodal(Ti,Z1,K2,Z2,ϕ)
println("T spinodal ", Tc)
Tλ=Tlambda(Tc,Z1,K2,Z2,ϕ) #temperatura de línea λ en phia
println("T lambda ", Tλ)
Ta=arrestoHSDY(ϕ,Z1,K2,Z2,Tλ) #temperatura de arresto en phia
println("T arresto ", Ta)
dif=Ta-Tλ
println("Dif ", dif)
difrel=dif/Tλ
println("Dif rel ", difrel)
difpor=difrel*100
println("Dif por ", difpor)
#=
Z1=10
Ta=0.07877156707102059
Tλ=0.07840066524797679
Diferencia Ta-Tλ
dif=0.0003709018230438066
Diferencia relativa dif/Tλ
difrel=0.004730850457335604
Diferencia porcentual difrel*100
difpor=0.4730850457335604
=#

Tsp= 0.012683202568887502
T spinodal 0.012683202568887502
Tλ= 0.012683445644079475
T lambda 0.012683445644079475
γ= 5.615802796626153e8 T= 0.06268344564407947
γ= 1.4615204492489785e13 T= 0.03768344564407947
γ= 4.794000433863269e10 T= 0.025183445644079476
γ= 5.258629132859932e9 T= 0.018933445644079477
γ= 8.773123005578455e11 T= 0.015808445644079475
γ= 0.27659249286520904 T= 0.014245945644079475
γ= 0.8842541954589491 T= 0.015027195644079474
γ= 4.001708138833536e8 T= 0.015417820644079474
γ= 3.1411113487528186e9 T= 0.015222508144079473
γ= 4.172148217525419e8 T= 0.015124851894079474
γ= 3.315480758440593e14 T= 0.015076023769079475
γ= 1.0076475406992402 T= 0.015051609706579474
γ= 1.1930322648313845 T= 0.015063816737829475
γ= 1.7714158803068954e11 T= 0.015069920253454475
γ= 1.98169075472876e11 T= 0.015066868495641976
γ= 2.0101356450167885e8 T= 0.015065342616735726
γ= 1.1111973465072378e14 T= 0.0150645796772826
γ= 5.417033284926718e8 T= 0.015064198207556037
γ= 3.375044763917173e12 T= 0.01506400

In [18]:
#Z1=100
Z1=100
Tc=Tspinodal(Ti,Z1,K2,Z2,ϕ)
println("T spinodal ", Tc)
Tλ=Tlambda(Tc,Z1,K2,Z2,ϕ) #temperatura de línea λ en phia
println("T lambda ", Tλ)
Ta=arrestoHSDY(ϕ,Z1,K2,Z2,Tλ) #temperatura de arresto en phia
println("T arresto ", Ta)
dif=Ta-Tλ
println("Dif ", dif)
difrel=dif/Tλ
println("Dif rel ", difrel)
difpor=difrel*100
println("Dif por ", difpor)
#=
Z1=100
Ta=0.008132440444409847
Tλ=0.00781425222414732
Diferencia Ta-Tλ
dif=0.00031818822026252747
Diferencia relativa dif/Tλ
difrel=0.04071895955434785
Diferencia porcentual difrel*100
difpor=4.071895955434785
=#

Tsp= 0.0011645486000242092
T spinodal 0.0011645486000242092
Tλ= 0.0011645708353506781
T lambda 0.0011645708353506781
γ= 4.838489625443588e15 T= 0.051164570835350684
γ= 5.985974786365234e13 T= 0.026164570835350683
γ= 3.012381402792995e17 T= 0.01366457083535068
γ= 9.412011417979202e15 T= 0.007414570835350679
γ= 5.3919531804480915e9 T= 0.004289570835350679
γ= 1.8486605733298828e11 T= 0.0027270708353506782
γ= 1.1583324961686297e8 T= 0.0019458208353506782
γ= 3.3728917969514804e9 T= 0.001555195835350678
γ= 0.10766301439905014 T= 0.001359883335350678
γ= 0.3533657178509952 T= 0.001457539585350678
γ= 1.2810989252949893e8 T= 0.001506367710350678
γ= 0.6593616998967872 T= 0.0014819536478506779
γ= 6.338709599483339e8 T= 0.001494160679100678
γ= 8.727133015317883e9 T= 0.001488057163475678
γ= 1.3337417766884497e12 T= 0.001485005405663178
γ= 8.613925449088168e11 T= 0.0014834795267569278
γ= 0.7027006960801971 T= 0.0014827165873038028
γ= 0.7404090066341681 T= 0.0014830980570303652
γ= 9.820935758999775e11

In [ ]:

difrel1=0.004730850355211999
difrel3=0.04071895955434785

Ti=10 #Temperatura inicial (T*) donde quieres empezar el barrido
phia=0.15 #Fracción en volumen en donde se ahce el análiss de arresto
#SpinodalHSDY(Ti,Z1,K2,Z2)
max=5
i=1
m=1
j=1
for m=1:max
    Z1=2.5+m*0.5
    for j=2.5+1:max
        Z2=2.5+j*0.5
        for i=1:max
            K2=i*0.5
            Tλ=Tlambda(Ti,Z1,K2,Z2,phia) #temperatura de línea λ en phia
            Ta=arrestoHSDY(phia,Z1,K2,Z2,Tλ) #temperatura de arresto en phia

            dif=Ta-Tλ #diferencia en temperaturas de lambda y arresto en phia

            println(Z1," ",K2," ",Z2," ",dif," ",Ta," ",Tλ)
        end
    end
end

In [58]:
#=[0.1, 0.15000000000000002, 0.2, 0.25, 0.3]
γ= 5.950268152101019 T*: 0.35900000000000004
ϕ= 0.1 T* arresto: 0.35900000000000004
γ= 2.1576941802318994 T*: 0.3663
ϕ= 0.15000000000000002 T* arresto: 0.3663=#

In [10]:
function Zcontraion(Z2,Zmacro,par)
   Zcontra = -(par[4]*(Z2)^2)/(24*par[5]*Zmacro) 
   return Zcontra
end

function X(i,carga,Γ,par)           
    X2 = (par[1]*σ[i]*carga)/(1 + Γ*σ[i])
    X3 = (par[1]*(σ[i]^3))/(1 + Γ*σ[i])
    X4 = (1/(1+ Γ*σ[i]))*(carga - par[3]*(σ[i]^2)*(X2/(1 + par[3]*X3)))
    return X4
end

function Kc(Z2,Zmacro,Γ,LBo,par)
    Um = ((3/(par[2]*(0.5*Z2)^3))*par[5])+((1/(0.5*Z2))*((X(1,Zmacro,Γ,par)/Zmacro)-1))
    K = ((LBo)^(0.5))*Zmacro*(cosh(0.5*Z2)+Um*(0.5*Z2*cosh(0.5*Z2)-sinh(0.5*Z2)))
    Kc = K^2
    return Kc
end

function Cero_A(Z2,K2,par)
    npasos=20000
    Zstep=0.001
    Zmacro=0
    Zcontra=0
    GammaFuera=0
    MargenB=K2*exp(Z2)-0.03
    MargenA=K2*exp(Z2)+0.03
    flag=0
    for j=1:npasos
        Zm=0.001+j*Zstep
        #Zm=1
        Zs=Zcontraion(Z2,Zm,par)
        ρs=-par[1]*Zm/Zs
        LBi = 1/(par[4]*σ[1])
        function Fg(Γ)
            D = 0
            D = ρs*(Zs^2) + par[1]*(X(1,Zm,Γ,par))^2 
            Fg = (Γ^2) - π*LBi*σ[1]*D
            return Fg
        end
        Γp=find_zero(Fg, 1)
        #println("Zmacro: ",Zm, ", Zcontra: ",Zs, ", Γ: ",Γp)
        Km=Kc(Z2,Zm,Γp,LBi,par)
        println(MargenB," ",Km," ",MargenA)
        if Km>=MargenB && Km<=MargenA
            Zmacro=Zm
            Zcontra=Zs
            GammaFuera=Γp
            flag=flag+1
            #println(Km," ",AY)
            #println("La carga de los macroiones equivalente es ", Zm)
            #println("La carga de los contraiones equivalente es ", Zs)
        end
        if flag==1
            break
        end
    end 
      
    return Zmacro,Zcontra,GammaFuera
end

function Parametros(ϕ,Z2,K2,TMP)
    parMP = zeros(5,1)
    parG = zeros(7,1)
    parMP[1] = 6*ϕ/(π*(σ[1]^3)) #Densidad de las macropartículas
    parMP[2] = 1-(π/6)*parMP[1]*σ[1]^3 #Δ
    parMP[3] = π/(2*parMP[2]) #c
    parMP[4] = TMP #Temperatura reducida como 1/LB*z1*z2
    parMP[5] = ϕ #Fracción en volumen de las macropartículas
    ZH=Cero_A(Z2,K2,parMP)
    parG[1] = ZH[1] #Carga macropartículas en unidades de e
    parG[2] = ZH[2] #Carga contraiones en unidades de e
    #println("Cargas: ", parG[1]," ",parG[2])
    parG[3] = parG[1]/parG[2] #Cociente de cargas
    parG[4] = -parMP[1]*parG[3] #Densidad reducida de los contraiones                                                   
    parG[5] = abs(parG[1]*parG[2]) #Valor absoluto del producto de cargas
    parG[6] = 1/(parMP[4]*σ[1]) #Longitud de Bjerrum MP
    parG[7] = ZH[3] #Valor de Γ
    return parMP,parG
end

function CHS(k,eta)
    η=(eta-(1.0/16.0)*eta^2)
    σw=(η/eta)^(1.0/3.0)
    k=k*σw
    
    #Coeficientes para la c(r) propuesta por Wertheim
    α=-((1.0+2.0*η)^2.0)/(1.0-η)^4.0
    β=6.0*η*(((1.0+0.5*η)^2.0)/((1.0-η)^4.0))
    δ=(η/2.0)*α
    
    #Componentes de la C(k) (transformada de la c(r) propuesta por Wertheim)
    i1=(sin(k)-k*cos(k))/k^3.0
    i2=((-(k^2.0) + 2.0)*cos(k) + 2.0*k*sin(k)-2.0)/k^4.0
    i3=((-(k^4.0) + 12.0*(k^2.0) - 24)*cos(k) +(4.0*(k^3.0)-24.0*k)*sin(k) + 24.0)/k^6.0

    #Definiendo el desarrollo en serie de las i1,i2,i3 para k pequeñas
    ip1=(1.0/3.0)-((k^2.0)*(1.0/30.0))
    ip2=(1.0/4.0)-((k^2.0)*(1.0/36.0))
    ip3=(1.0/6.0)-((k^2.0)*(1.0/48.0))

    #C(K) 
    if k<0.075
        ck=24.0*η*(α*ip1+β*ip2+δ*ip3)
        else 
        ck=24.0*η*(α*i1+β*i2+δ*i3)
    end
    
    return ck

end

function Qk(k,vecHS,i,j)
    ρ=[vecHS[1],vecHS[2]]
    
    ζ2 = ρ[1]*(σ[1]^2)*π/6
    ζ3 = ρ[1]*(σ[1]^3)*π/6

    Sr =(σ[i] - σ[j])/2 
    R = (σ[i] + σ[j])/2
    ai = (1- ζ3 + 3*σ[i]*ζ2)/((1 - ζ3)^2)
    bi = -(3/2)*(σ[i]^2)*ζ2/((1-ζ3)^2)

    if i==j
    delta=1 
    else
    delta=0
    end 

    pR=(ai/2)*exp(im*k*R)*((2-2*im*k*R-(k^2)*(R^2))/(im*k)^3) -(ai/2)*(R^2)*(exp(im*k*R)/(im*k)) +bi*exp(im*k*R)*((im*k*R - 1)/((im*k)^2)) - bi*R*(exp(im*k*R)/(im*k))

    pS=(ai/2)*exp(im*k*Sr)*((2-2*im*k*Sr-(k^2)*(Sr^2))/(im*k)^3) -(ai/2)*(R^2)*(exp(im*k*Sr)/(im*k)) +bi*exp(im*k*Sr)*((im*k*Sr - 1)/((im*k)^2)) - bi*R*(exp(im*k*Sr)/(im*k))

   # pRSp=(-(1/6)*ai)*(2*R^3+Sr*(Sr^2-3*R^2)) +(-(1/8)*im*ai*k)*(R^4+Sr^2*(Sr^2-2*R^2)) +((1/60)*ai*(k^2))*(2*R^5-(Sr^3)*(5*R^2-3*Sr^2))+(-(1/2)*bi)*(R^2+Sr*(Sr-2*R))-((1/6)*im*bi*k)*(R^3+(Sr^2)*(3*R-2*Sr))
     #   +((1/24)*bi*k^2)*(R^4-(Sr^3)*(4*R-3*Sr))                                     
    
    #kp=abs(k)
    #if kp<0.075
    #    Q = delta - 2*π*sqrt(ρ[i]*ρ[j])*pRSp
    #else
        Q = delta - 2*π*sqrt(ρ[i]*ρ[j])*(pR-pS)
    #end
    
    return Q
end

function Q(k,vecHS)    
    Q = [[Qk(k,vecHS,1,1),Qk(k,vecHS,2,1)]  [Qk(k,vecHS,1,2),Qk(k,vecHS,2,2)] ]
    return Q
end

function QT(k,vecHS)   
    QT=transpose(Q(-k,vecHS))
    return QT
end

function Shs(k,vecHS)  
    S=inv(QT(k,vecHS)*Q(k,vecHS))
    return real(S)
end

function ρChs(k,vecHS)  
    I=[[1,0] [0,1]]
    C=(Shs(k,vecHS)- I)/(Shs(k,vecHS))
    return C
end
    
function N(i,Z,par) 
    XX = X(i,Z,par[7],par)
    N=-(par[3]*σ[i]*par[8])-(XX*par[7])
    return N
end

function ckelec(k,par,i,j)
    ρ=[par[1],par[6]]
    z=[par[4],par[5]]
    p=par[3]
    LB=par[2]
    Γg=par[7]
    Acte = par[8]
    Xi = X(i,z[i],Γg,par)
    Xj = X(j,z[j],Γg,par)
    Ni = N(i,z[i],par)
    Nj = N(j,z[j],par)
    
    σij=(σ[i] + σ[j])/2
    σmin= (σ[j] - σ[i])/2 
   
    #Constante de 0 a  σmin
    c1 = -2*(-z[i]*Nj + Xi*(Ni + Γg*Xi) - (σ[i]/3)*((Ni + Γg*Xi)^2))    

    #Constantes de  σmin a σij
    c2 = (σ[i]-σ[j])*(((Xi + Xj)/4)*(Ni + Γg*(Xi-Xj) - Nj) - ((σ[i]-σ[j])/16)*((Ni + Γg*(Xi+Xj) + Nj)^2 - 4*Ni*Nj)) 
    c3 = -((Xi-Xj)*(Ni-Nj) + (Xi^2 + Xj^2)*Γg + (σ[i] + σ[j])*Ni*Nj -(1/3)*(σ[i]*(Ni + Γg*Xi)^2 + σ[j]*(Ni + Γg*Xj)^2))
    c4 = ((-p*Acte)*(Xi+Xj) + Ni*Nj -(1/2)*(((p*Acte)^2)*(σ[i]^3 + σ[j]^3)))
    c5 = (1/3)*(p*Acte)^2

    #Constante de  σij a infinito         
    c6= -z[i]*z[j]        

    I = (sin(k*σmin)-k*σmin*cos(k*σmin))/(k^2)
    II = (cos(k*σmin) - cos(k*σij))/k
    III = (sin(k*σij)-k*σij*cos(k*σij) - sin(k*σmin) +k*σmin*cos(k*σmin))/(k^2)
    IV = ((2 - (k^2)*(σij^2))*cos(k*σij) + 2*k*σij*sin(k*σij) -(2 - (k^2)*(σmin^2))*cos(k*σmin) - 2*k*σmin*sin(k*σmin))/(k^3)
    V = (4*k*σij*((k^2)*(σij^2)-6)*sin(k*σij) - ((k^4)*(σij^4) - 12*(k^2)*(σij^2) + 24)*cos(k*σij) - 4*k*σmin*((k^2)*(σmin^2)-6)*sin(k*σmin) + ((k^4)*(σmin^4) - 12*(k^2)*(σmin^2) + 24)*cos(k*σmin))/(k^5)
    VI = cos(k*σij)/k
    
    #Ip = ((σmin)^3)/3
    #IIp = ((σij)^2-(σmin)^2)/2
    #IIIp = ((σij)^3-(σmin)^3)/3
    #IVp = ((σij)^4-(σmin)^4)/4
    #Vp = ((σij)^6-(σmin)^6)/6
    #VIp = 1/k^2 - (σij^2)/2

   # if k<0.075
    #    c = sqrt(ρ[i]*ρ[j])*(4*π*LB)*(c1*Ip + c2*IIp + c3*IIIp + c4*IVp + c5*Vp + c6*VIp)
     #   else 
        c = sqrt(ρ[i]*ρ[j])*(4*π*LB/k)*(c1*I + c2*II + c3*III + c4*IV + c5*V + c6*VI)
    #end

        #c = sqrt(ρ[i]*ρ[j])*(4*π*LB/k)*(c1*I + c2*II + c3*III + c4*IV + c5*V + c6*VI)
    
    return c
end
  
function ρCelec(k,par)
    vecMP = [par[1],par[11],par[3],par[6],par[7],par[9],par[12],0]
    #=
        vecMP[1] #Densidad de las macropartículas                  
        vecMP[2] #Longitud de Bjerrum MP
        vecMP[3] #c
        vecMP[4] #Carga macropartículas en unidades de e
        vecMP[5] #Carga contraiones en unidades de e
        vecMP[6] #Densidad reducida de los contraiones
        vecMP[7] #Parámetro Γ
        vecMP[8] #Constante AGAMMA
    =#
    Γsol = vecMP[7]
    function AGAMMA(i,Z,Γ,par)           
        X2 = (par[1]*σ[i]*Z)/(1 + Γ*σ[i])
        X3 = (par[1]*(σ[i]^3))/(1 + Γ*σ[i])
        X4 = X2/(1 + par[3]*X3)
        return X4
    end
    
    vecMP[8] = AGAMMA(1,vecMP[4],Γsol,vecMP) 

    B = [[ckelec(k,vecMP,1,1),ckelec(k,vecMP,2,1) ] [ckelec(k,vecMP,1,2),ckelec(k,vecMP,2,2) ] ]
    return B
end

function ρC(k,par)    
    vecHS = [par[1],par[9]]
    B = ρChs(k,vecHS) + ρCelec(k,par)
    return B
end

function Cefec(k,par)
    A = ρC(k,par)[1,1]+((ρC(k,par)[1,2]*ρC(k,par)[2,1])/(1-ρC(k,par)[2,2]))
    return A
end

function C(k,Z1,K1,par)
    k=k*σ[1]
    AY=(24*par[5]*K1)*((Z1*sin(k)+k*cos(k))/(k*(Z1^2+k^2)))
    
    CTotal=Cefec(k,par)+AY
    
    return CTotal
end

function CDSS(k,par)
    K1=par[1]
    Z1=par[2]
    K2=par[3]
    Z2=par[4]
    η=par[5]
    
    AY=(24*η*K1)*((Z1*sin(k)+k*cos(k))/(k*(Z1^2+k^2)))
    RY=(24*η*K2)*((Z2*sin(k)+k*cos(k))/(k*(Z2^2+k^2)))
    
    AYp=(24*η*K1)*((Z1+1)/(Z1^2))
    RYp=(24*η*K2)*((Z2+1)/(Z2^2))

    if k<0.075
        CTotal=CHS(k,η)+AYp-RYp
    else
        CTotal=CHS(k,η)+AY-RY
    end
    
    return CTotal
end

function SHSAYRYSS(Tr,Z1,K2,Z2,phi)
    K1=1/Tr
    parSS = [K1,Z1,K2,Z2,phi]
    CSS=zeros(NKMAX,1)
    SSS=zeros(NKMAX,1)
    for i=1:NKMAX
        CSS[i]=CDSS(kv[i],parSS)
        SSS[i]=1.0/(1.0 - CSS[i])
    end

    SSk=save_data("SKSS_$(Tr)_$(Z1)_$(K2)_$(Z2)_$phi.dat", [kv SSS])
    #SSk=save_data("SKHSAY_$(Tr)_$(Z1)_$phi.dat", [kv SSS])

end

function SHSAYRY(TMP,Tr,Z1,K2,Z2,phi)
    K1=1/Tr
    Ceff=zeros(NKMAX,1)
    Seff=zeros(NKMAX,1)
    #TMP = 0.1
    vi = Parametros(phi,Z2,K2,TMP)
    vec1 = vi[1]
    vec2 = vi[2]
    vecpar = [vec1[1],vec1[2],vec1[3],vec1[4],vec1[5],vec2[1],vec2[2],vec2[3],vec2[4],vec2[5],vec2[6],vec2[7]]
    #println(vecpar)
#=
    par[1] #Densidad de las macropartículas
    par[2] #Δ
    par[3] #c
    par[4] #Temperatura reducida como 1/LB
    par[5] #Fracción en volumen de las macropartículas
    par[6] #Carga macropartículas en unidades de e
    par[7] #Carga contraiones en unidades de e
    par[8] #Cociente de cargas
    par[9] #Densidad reducida de los contraiones                                                   
    par[10] #Valor absoluto del producto de cargas
    par[11] #Longitud de Bjerrum MP
    par[12] #Valor de Γ 
=#

    for i=1:NKMAX
        Ceff[i]=C(kv[i],Z1,K1,vecpar)
        Seff[i]=1.0/(1.0 - Ceff[i])
    end

    SKeff=save_data("SK_$(Tr)_$(Z1)_$(K2)_$(Z2)_$phi.dat", [kv Seff])

end

function SpinodalHSAYRYSS(Ti,Z1,K2,Z2)
    PHIMAX = 120
    phiv = [ 0.005*i for i=1:PHIMAX]
    SSLO=zeros(PHIMAX,1)
    Sesp=0
    println("Curva espinodal HS+RY+AY S-S")
    tol=1.0e-7
    
    for i=1:PHIMAX
        phitest=i*0.005
        vi = [0,Z1,K2,Z2,phitest]
        errortemp=1.0
        limS=Ti
        limI=1.0e-5
        Ttest=1
        
        while errortemp>tol
            Ttest=(limS+limI)*0.5
            K1=1/Ttest
            vi[1]=K1
            Sesp=1.0/(1.0 - CDSS(0,vi)) 

            if Sesp<=0.0
                limI=Ttest
            else
                limS=Ttest
            end

            errortemp=abs(1-limS/limI)
        end
        
        #println("T= ",Ttest," ϕ: ",phitest," S: ",Sesp, " error: ",errortemp)
        SSLO[i]=Ttest

    end
    
    SpinodalLO= save_data("SpSS_$(Z1)_$(K2)_$(Z2).dat", [phiv SSLO])
end

function LambdaLineSS(Ti,Z1,K2,Z2)
    PHIMAX = 120
    phiv = [ 0.005*i for i=1:PHIMAX]
    SSλLO=zeros(PHIMAX,1)
    Sesp=zeros(100,1)
    println("Lambda line HS+RY+AY S-S")
    tol=1.0e-7
    
    for i=1:PHIMAX
        phitest=i*0.005
        vi = [0,Z1,K2,Z2,phitest]
        errortemp=1.0
        limS=Ti
        limI=1.0e-5
        Ttest=1
        
        while errortemp>tol
            Ttest=(limS+limI)*0.5
            K1=1/Ttest
            vi[1]=K1
            flag=0
            
            for m=1:100
                Sesp[m]=1.0/(1.0 - CDSS(kv[m],vi))
                if Sesp[m]<=0.0
                    flag=flag+1
                    break
                end
            end
            
            if flag>=1
                limI=Ttest
            else
                limS=Ttest
            end

            errortemp=abs(1-limS/limI)
        end
        
        SSλLO[i]=Ttest

    end
    
    LambdaLO= save_data("LSS_$(Z1)_$(K2)_$(Z2).dat", [phiv SSλLO])
end

function Tcritica(phi,S_HS,z1,z2,k2)
      Tab = 24*phi*S_HS
      Tab = 1/Tab
      fac1 = (z1 + 1) / (z1*z1)
      fac2 = (z2 + 1) / (z2*z2)
      Ts = fac1 / (Tab + k2*fac2)
    return Ts
end

function SpinodalHSAYRY(Ti,Z1,K2,Z2)
    PHIMAX = 120
    phiv = [ 0.005*i for i=1:PHIMAX]
    SSDY=zeros(PHIMAX,1)
    Sesp=0
    TMP = 0.5
    println("Curva espinodal HS+RY+AY")
    tol=1.0e-7
    
    for i=1:PHIMAX
        phitest=i*0.005
        vi = [0,Z1,K2,Z2,phitest]
        errortemp=1.0
        limS=Ti
        limI=1.0e-5
        Ttest=1
        
        while errortemp>tol
            Ttest=(limS+limI)*0.5
            K1=1/Ttest
            vi = Parametros(phitest,Z2,K2,TMP)
            vec1 = vi[1]
            vec2 = vi[2]
            vecpar = [vec1[1],vec1[2],vec1[3],vec1[4],vec1[5],vec2[1],vec2[2],vec2[3],vec2[4],vec2[5],vec2[6],vec2[7]]
            Sesp=1.0/(1.0 - C(kv[1],Z1,K1,vecpar))

            if Sesp<=0.0
                limI=Ttest
            else
                limS=Ttest
            end

            errortemp=abs(1-limS/limI)
        end
        
        println("T= ",Ttest," ϕ: ",phitest," S: ",Sesp, " error: ",errortemp)
        SSDY[i]=Ttest

    end
    
    SpinodalDY= save_data("Sp_$(Z1)_$(K2)_$(Z2).dat", [phiv SSDY])
end

function LambdaLine(Ti,Z1,K2,Z2)
    PHIMAX = 120
    phiv = [ 0.005*i for i=1:PHIMAX]
    SSλDY=zeros(PHIMAX,1)
    Sesp=zeros(100,1)
    TMP = 0.5
    println("Lambda line HS+RY+AY")
    tol=1.0e-7
    
    for i=1:PHIMAX
        phitest=i*0.005
        vi = Parametros(phitest,Z2,K2,TMP)
        vec1 = vi[1]
        vec2 = vi[2]
        vecpar = [vec1[1],vec1[2],vec1[3],vec1[4],vec1[5],vec2[1],vec2[2],vec2[3],vec2[4],vec2[5],vec2[6],vec2[7]]
        errortemp=1.0
        limS=Ti
        limI=1.0e-5
        Ttest=1
        
        while errortemp>tol
            Ttest=(limS+limI)*0.5
            K1=1/Ttest
            flag=0
            
            for m=1:100
                Sesp[m]=1.0/(1.0 - C(kv[m],Z1,K1,vecpar)) 
                if Sesp[m]<=0.0
                    flag=flag+1
                    break
                end
            end
            
            if flag>=1
                limI=Ttest
            else
                limS=Ttest
            end

            errortemp=abs(1-limS/limI)
        end
        
        SSλDY[i]=Ttest

    end
    
    LambdaDY= save_data("L_$(Z1)_$(K2)_$(Z2).dat", [phiv SSλDY])
end

function λ(k)
    kmin=2*π*1.305/σ[1]
    lam=1/(1+(k/kmin)^2)
    λ=lam
    return λ
end

function γ(γ1,Si,Hi,rho)  
    suma1=0.0
    i=0

    for i=1:NKMAX
        A=(λ(kv[i])^2)*(Hi[i])^2
        B=(λ(kv[i])*Si[i])+((kv[i]^2)*γ1)
        C=λ(kv[i])+((kv[i]^2)*γ1)
        
        suma1 = suma1 + (kv[i]^4)*(A/(B*C))
        
    end
        #=recuerda que la renormalizacion es interna al factor de estructura, 
        y verlet weiss dice que la estructura correcta esta desfasada en terminos
        de la dimension de particula, lo que implica que tienes que renormalizar
        tanto k como la eta pero solo en s((k,eta)=#
    int1=suma1*dk/(6*rho*π^2)
    γ1= 1/int1
    
    return γ1
end

function criterioγ(Svec,Hvec,rho)
    i=0
    inf=1.0e8 
    tol=1.0e-7
    errorγ=1.0
    γtest=1.0e-6
    
    while tol<errorγ
        i=i+1  
        γp=γ(γtest,Svec,Hvec,rho) 
        γ1=γp
        errorγ=abs(1-γ1/γtest)
        γtest=γ1

        #println("γ= ",γtest," error= ",errorγ)

        if (inf<γ1) break
        end 
        
        if i>100000
            break
        end
        
    end 
    
    return γtest
end

function arrestoHSAYRYSS(TY,Z1,K2,Z2)
    K1=1/TY
    errorphi= 1.0
    tol=1.0e-7
    inf=1.0e8 
    TMP=0.5
    Carr=zeros(NKMAX,1)
    Sarr=zeros(NKMAX,1)
    Harr=zeros(NKMAX,1)
    limS=0.35
    limI=0.27
    phitest=0.38
    γ1=0
    i=0
    
    while errorphi>tol
        i=i+1
        phitest=(limS+limI)*0.5
        rhotest=6*phitest/(π*(σ[1]^3))
        vi = [K1,Z1,K2,Z2,phitest]
        for i=1:NKMAX
            Carr[i]=CDSS(kv[i],vi)
            Sarr[i]=1.0/(1.0 - Carr[i])
            Harr[i]=Sarr[i]-1.0
        end

        γ1=criterioγ(Sarr,Harr,rhotest)

        if γ1<inf
            limS=phitest
        else
            limI=phitest
        end

        errorphi=abs(1-limS/limI)
        
        #println("γ= ",γ1," ϕ: ",phitest)
    end

    return phitest
end

function arrestoHSAYRYSSNOBIS(phitest,Z1,K2,Z2)
    tol=1.0e-7
    inf=1.0e8 
    grande=20000
    Ttest=1
    Carr=zeros(NKMAX,1)
    Sarr=zeros(NKMAX,1)
    Harr=zeros(NKMAX,1)
    γ1=grande+1
    i=0
    
    while γ1>grande
        i=i+1
        Ttest=0.1-0.00001*i
        K1=1/Ttest
        rhotest=6*phitest/(π*(σ[1]^3))
        vi = [K1,Z1,K2,Z2,phitest]
        for i=1:NKMAX
            Carr[i]=CDSS(kv[i],vi)
            Sarr[i]=1.0/(1.0 - Carr[i])
            Harr[i]=Sarr[i]-1.0
        end

        γ1=criterioγ(Sarr,Harr,rhotest)
        
    end
    
    println("γ= ",γ1," T*: ",Ttest)

    return Ttest
end



function arrestoHSAYRY(TY,Z1,K2,Z2)
    K1=1/TY
    errorphi= 1.0
    tol=1.0e-7
    inf=1.0e8 
    TMP=0.5
    Carr=zeros(NKMAX,1)
    Sarr=zeros(NKMAX,1)
    Harr=zeros(NKMAX,1)
    limS=0.33
    limI=0.26
    phitest=0.4
    γ1=0
    i=0
    
    while errorphi>tol
        i=i+1
        phitest=(limS+limI)*0.5
        vi = Parametros(phitest,Z2,K2,TMP)
        vec1 = vi[1]
        vec2 = vi[2]
        vecpar = [vec1[1],vec1[2],vec1[3],vec1[4],vec1[5],vec2[1],vec2[2],vec2[3],vec2[4],vec2[5],vec2[6],vec2[7]]
        for i=1:NKMAX
            Carr[i]=C(kv[i],Z1,K1,vecpar)
            Sarr[i]=1.0/(1.0 - Carr[i])
            Harr[i]=Sarr[i]-1.0
        end

        γ1=criterioγ(Sarr,Harr,vec1[1])

        if γ1<inf
            limS=phitest
        else
            limI=phitest
        end

        errorphi=abs(1-limS/limI)
        
        #println("γ= ",γ1," ϕ: ",phitest)
    end

    return phitest
end


function DiagArresto(Ti,Tf,Z1,K2,Z2)
    Ttotal=Int((Ti-Tf)/0.01)
    AD=zeros(Ttotal,1)
    Tprueba = [ Tf+0.01*i for i=1:Ttotal]
    
    println("T inicio: ", Tprueba[Ttotal]," T final: ",Tprueba[1])
    
    for i=1:Ttotal
        j=Ttotal-i+1
        AD[i]=arrestoHSAYRY(Tprueba[j],Z1,K2,Z2)
        println("T*= ",Tprueba[j]," ϕ arresto: ",AD[i])
    end

    ArDi= save_data("Arresto_$(Z1)_$(K2)_$(Z2).dat", [Tprueba AD])   

end

function DiagArrestoSS(Ti,Tf,Z1,K2,Z2)
    Ttotal=Int(round((Ti-Tf)/0.001))
    AD=zeros(Ttotal,1)
    Tprueba = [ Tf+0.001*i for i=1:Ttotal]
    
    for i=1:Ttotal
        j=Ttotal-i+1
        AD[i]=arrestoHSAYRYSS(Tprueba[j],Z1,K2,Z2)
        println("T*= ",Tprueba[j]," ϕ arresto: ",AD[i])
    end

    ArDi= save_data("ArrestoSS_$(Z1)_$(K2)_$(Z2).dat", [Tprueba AD])   
    
end

function DiagArrestoSSNOBIS(phiI,phiF,Z1,K2,Z2)
    step=0.001
    Phitotal=Int(round((phiF-phiI)/step))
    AD=zeros(Phitotal,1)
    PhiPrueba = [ phiI+step*i for i=1:Phitotal]
    println(PhiPrueba)
    for i=1:Phitotal
        AD[i]=arrestoHSAYRYSSNOBIS(PhiPrueba[i],Z1,K2,Z2)
        println("ϕ= ",PhiPrueba[i]," T* arresto: ",AD[i])
    end

    ArDi= save_data("ArrestoSSNOBIS_$(Z1)_$(K2)_$(Z2).dat", [PhiPrueba AD])   
    
end


DiagArrestoSSNOBIS (generic function with 1 method)